In [2]:
# 引入套件

import pandas as pd
import numpy as np
import csv
import jieba
import random
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from tqdm import tqdm

In [2]:
# 引入品類訓練資料集

catdata = pd.read_csv('整理的品類資料集/cleaned_train.csv')
catdata = catdata[['name', 'product']]

In [3]:
catdata

,name,product
0,r視舒坦單支裝人工淚液點眼液,人工淚液
1,優麗舒點眼液,人工淚液
2,r視舒坦單支裝人工,人工淚液
3,rohto乾眼淚液,人工淚液
4,r參天柔潤人工淚液點眼液,人工淚液
...,...,...
92294,保證bb霜 隔離修容bb美顏霜 粉底液 遮瑕 修容,BB霜
92295,庶務客 pony effect 遮瑕氣墊粉餅 粉底 神防護 氣墊 bb霜 氣墊粉餅,粉餅
92296,kose 高絲雪肌精透亮煥白cc霜雪肌精淨透潔顏油n 正統公司貨,CC霜
92297,drcink 達特聖克女神絲絨柔焦組cc霜花蜜露潔面露,CC霜


In [5]:
# Jieba處理，存於cutname欄位

catdata['cutname'] = ''
for i in range(len(catdata)):
    catdata['cutname'][i] = ' '.join(jieba.cut_for_search(catdata['name'][i]))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/jieba.cache
Loading model cost 0.451 seconds.
Prefix dict has been built successfully.


In [6]:
catdata

,name,product,cutname
0,r視舒坦單支裝人工淚液點眼液,人工淚液,r 視 舒坦 單支裝 人工 淚 液點 眼液
1,優麗舒點眼液,人工淚液,優麗舒點 眼液
2,r視舒坦單支裝人工,人工淚液,r 視 舒坦 單支裝 人工
3,rohto乾眼淚液,人工淚液,rohto 乾眼 淚液
4,r參天柔潤人工淚液點眼液,人工淚液,r 參天 柔潤 人工 淚 液點 眼液
...,...,...,...
92294,保證bb霜 隔離修容bb美顏霜 粉底液 遮瑕 修容,BB霜,保證 bb 霜 隔離 修容 bb 美顏霜 粉底 粉底液 遮 瑕 修容
92295,庶務客 pony effect 遮瑕氣墊粉餅 粉底 神防護 氣墊 bb霜 氣墊粉餅,粉餅,庶務客 pony effect 遮 瑕 氣墊 粉餅 粉底 神防護 氣...
92296,kose 高絲雪肌精透亮煥白cc霜雪肌精淨透潔顏油n 正統公司貨,CC霜,kose 高絲 雪肌精 透亮 煥白 cc 霜 雪肌精 淨透潔 顏油 n 正統 公司 貨
92297,drcink 達特聖克女神絲絨柔焦組cc霜花蜜露潔面露,CC霜,drcink 達特聖克 女神 絲絨柔 焦組 cc 霜花 蜜露 潔面露


In [11]:
# 切分資料函式，82分

def Splitdata(data, ratio=0.8):
    train_x = [] # 儲存訓練集產品名稱
    train_y = [] # 儲存訓練集產品品類
    test_x = [] # 儲存測試集產品名稱
    test_y = [] # 儲存測試集產品品類
    for uniname in data['product'].unique():
        # 以產品品類為區隔將資料82分
        matchdata = data[data['product'] == uniname]
        random.seed(1)
        matchdata = shuffle(matchdata)
        train_num = round(0.8*len(matchdata))
        
        for i in matchdata['cutname'][:train_num]:
            train_x.append(i)
        for i in matchdata['product'][:train_num]:
            train_y.append(i)
        
        for i in matchdata['cutname'][train_num:]:
            test_x.append(i)
        for i in matchdata['product'][train_num:]:
            test_y.append(i)
    
    return train_x, train_y, test_x, test_y

In [19]:
# 切分資料
x_train, y_train, x_test, y_test = Splitdata(catdata)

In [20]:
# 將算法與模型串聯，平滑參數設為0.01
model = make_pipeline(TfidfVectorizer(), MultinomialNB(alpha = 0.01))

In [21]:
# 執行訓練過程
model.fit(x_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB(alpha=0.01))])

In [22]:
# 計算正確率
model.score(x_test, y_test)

0.9034774130646733

In [24]:
# 預測測試集，並計算F1 Score

y_pred = model.predict(x_test)
f1_score(y_test,y_pred,average='macro')

0.8248601904680566

In [62]:
# 預測品類函式

def predict_cat(s, train=x_train, model=model):
    pred = model.predict([s])
    return pred[0]

In [57]:
# 讀入品類資料集
prddata = pd.read_csv('整理的品類資料集/cleaned_test.csv')
prddata = prddata[['name','clean_name', 'product']]
prddata['product'] = ''

In [59]:
# Jieba處理，存於cutname欄位

prddata['cutname'] = ''
for i in range(len(prddata)):
    prddata['cutname'][i] = ' '.join(jieba.cut_for_search(prddata['clean_name'][i]))

In [60]:
prddata

,name,clean_name,product,cutname
0,【施巴】嬌顏卸妝乳ha400ml,施巴嬌顏卸妝乳ha,,施巴嬌 顏卸妝乳 ha
1,艾瑪絲舒敏平衡洗髮精40,艾瑪絲舒敏平衡洗髮精,,艾瑪絲 舒敏 平衡 洗 髮 精
2,【bb love】高質感天絲tencel舖棉兩用兒童睡袋(多款可選),bb love高質感天絲tencel舖棉兩用兒童睡袋多款可選,,bb love 高質 感天絲 tencel 舖 棉 兩用 兒童 睡袋 多款 可選
3,lovita愛維他-緩釋型維生素c 1000mg(60錠),lovita愛維他緩釋型維生素c,,lovita 愛維 他 緩釋 型 維生素 c
4,111262674618 cm 12公分長效rifle風扇 1200轉,cm 長效rifle風扇 1200轉,,cm 長 效 rifle 風扇 1200 轉
...,...,...,...,...
108846,歐舒丹~橙花蘭花護手霜30ml,歐舒丹橙花蘭花護手霜,,歐舒丹 橙 花蘭花護 手霜
108847,高露潔 360度抗敏專家牙刷(1入),高露潔 360度抗敏專家牙刷,,高露潔 360 度 抗敏 專家 牙刷
108848,高露潔專業型超小刷頭密植軟毛牙刷6支裝【康是美】,高露潔專業型超小刷頭密植軟毛牙刷康是美,,高露潔 專業型 超小刷頭 密植 軟毛 牙刷 康是 美
108849,熊寶貝 沁藍海洋香柔軟衣物護衣精2+6組 (3.2lx2+1.84lx6包),熊寶貝 沁藍海洋香柔軟衣物護衣精2,,熊寶貝 沁藍 海洋 香柔軟 衣物 護衣 精 2


In [64]:
# 預測並填入品類欄位，若遇號碼將號碼填入

for i in tqdm(range(len(prddata))):
    if not prddata['cutname'][i].isnumeric():
        prddata['product'][i] = predict_cat(prddata['cutname'][i])
    else:
        prddata['product'][i] = prddata['cutname'][i]

100%|███████████████████████████████████| 108851/108851 [18:28<00:00, 98.19it/s]


In [65]:
prddata

,name,clean_name,product,cutname
0,【施巴】嬌顏卸妝乳ha400ml,施巴嬌顏卸妝乳ha,卸妝乳,施巴嬌 顏卸妝乳 ha
1,艾瑪絲舒敏平衡洗髮精40,艾瑪絲舒敏平衡洗髮精,洗髮精,艾瑪絲 舒敏 平衡 洗 髮 精
2,【bb love】高質感天絲tencel舖棉兩用兒童睡袋(多款可選),bb love高質感天絲tencel舖棉兩用兒童睡袋多款可選,睡袋,bb love 高質 感天絲 tencel 舖 棉 兩用 兒童 睡袋 多款 可選
3,lovita愛維他-緩釋型維生素c 1000mg(60錠),lovita愛維他緩釋型維生素c,維他命B,lovita 愛維 他 緩釋 型 維生素 c
4,111262674618 cm 12公分長效rifle風扇 1200轉,cm 長效rifle風扇 1200轉,電風扇,cm 長 效 rifle 風扇 1200 轉
...,...,...,...,...
108846,歐舒丹~橙花蘭花護手霜30ml,歐舒丹橙花蘭花護手霜,護手霜,歐舒丹 橙 花蘭花護 手霜
108847,高露潔 360度抗敏專家牙刷(1入),高露潔 360度抗敏專家牙刷,成人牙刷,高露潔 360 度 抗敏 專家 牙刷
108848,高露潔專業型超小刷頭密植軟毛牙刷6支裝【康是美】,高露潔專業型超小刷頭密植軟毛牙刷康是美,成人牙刷,高露潔 專業型 超小刷頭 密植 軟毛 牙刷 康是 美
108849,熊寶貝 沁藍海洋香柔軟衣物護衣精2+6組 (3.2lx2+1.84lx6包),熊寶貝 沁藍海洋香柔軟衣物護衣精2,柔軟精,熊寶貝 沁藍 海洋 香柔軟 衣物 護衣 精 2


In [68]:
# 匯出資料集

df = prddata[['name', 'product']]
df.to_csv('cat_test_v2(answer).csv', encoding="utf-8")

In [7]:
df.to_csv('cat_test_v2(answer)sig.csv', encoding="utf-8-sig")